In [1]:
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import COBYLA

from circuit_knitting.cutting.gate_and_wire_cutting.algorithms.cut_vqe import CutVQE


# Use a list of H2 observables from the PySCF driver
H2_op_list = [('II', -1.0523732457728596),
             ('IZ', 0.39793742484317934),
             ('XX', 0.18093119978423144),
             ('ZI', -0.39793742484317934),
             ('ZZ', -0.011280104256235268)]

# Problem specification. Hamiltonian for H2 at 0.735A interatomic distance.
H2_op = SparsePauliOp.from_list(H2_op_list)

# Create an ansatz
ansatz = EfficientSU2(H2_op.num_qubits, reps=1)

# Decompose into individual gates
circuit = ansatz.decompose(reps=3)

circuit

/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(


In [3]:
# See how many parameters are in our ansatz
num_params = circuit.num_parameters
print(f'Num parameters: {num_params}')

# List out the observables
observables = H2_op.paulis
print(f'Observables: {observables}')

Num parameters: 8
Observables: ['II', 'IZ', 'XX', 'ZI', 'ZZ']


In [4]:
# Set up the azure backend
from azure.quantum import Workspace
from azure.quantum.qiskit import AzureQuantumProvider

#workspace = Workspace(
#            resource_id = "/subscriptions/4ed1f7fd-7d9e-4c61-9fac-521649937e65/resourceGroups/Cutting/providers/Microsoft.Quantum/Workspaces/Cutting",
#            location = "eastus")
workspace = Workspace.from_connection_string("SubscriptionId=66c5a625-c800-4c9f-87e3-542d09f3ec6c;ResourceGroupName=Documentation;WorkspaceName=Accenture-Quantum-University;ApiKey=Q2AdKU9lwjIyFyHpU1S8qS7wiybCCZVXDiKSdlESE4ZJYM6c2BR_ycMvssxbVXot9_yHAw7pBvR9AZQUpz7uSQ;QuantumEndpoint=https://eastus.quantum.azure.com/;")
provider = AzureQuantumProvider(workspace)
backend = provider.get_backend("quantinuum.sim.h1-1sc")

usable_backends = ['quantinuum.sim.h1-1sc', 'quantinuum.sim.h1-2sc', 'quantinuum.sim.h1-1e', 'quantinuum.sim.h1-2e',
                   'quantinuum.qpu.h1-1', 'quantinuum.qpu.h1-2']


In [5]:
# VQE
from qiskit_algorithms.minimum_eigensolvers import VQE, AdaptVQE
from qiskit_algorithms.optimizers import SLSQP, COBYLA
from qiskit.circuit.library import EvolvedOperatorAnsatz
from qiskit.primitives import Estimator

def callback(eval_count, parameters, mean, std):
    print(f"Round num: {eval_count}, energy: {mean}, parameters: {parameters}")

In [6]:
# Define our estimator and optimizer
estimator = Estimator()
optimizer = COBYLA(maxiter=2)

In [7]:
from circuit_knitting.cutting.gate_and_wire_cutting.algorithms.cut_vqe import CutVQE
# Run VQE and print our results
vqe = CutVQE(estimator, circuit, optimizer, observables=observables, shots=2**12, max_subcircuit_width=1, max_cuts=9, num_subcircuits=[2], model='gurobi', num_samples=1500, callback=callback, backend='azure', azure_backend=backend)

Restricted license - for non-production use only - expires 2024-10-28
Set parameter TimeLimit to value 300
Set parameter Cutoff to value 1e+100
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11850H @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 31 rows, 21 columns and 74 nonzeros
Model fingerprint: 0x7d6ae72b
Variable types: 0 continuous, 21 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 1.0000000
Presolve removed 31 rows and 21 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 1 

Optimal solution found (to

In [8]:
with backend.open_session(name="Qiskit Session") as session:
    result = vqe.compute_minimum_eigenvalue(H2_op)

print(result)

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


AlgorithmError: 'The primitive job to evaluate the energy failed!'

In [9]:
# Do the same thing to compare againt not cutting at all
estimator = Estimator()
optimizer = COBYLA(maxiter=80)
no_cut_vqe = VQE(ansatz=circuit, optimizer=optimizer, estimator=estimator, callback=callback)
no_cut_result = no_cut_vqe.compute_minimum_eigenvalue(H2_op)
print(no_cut_result)

Round num: 1, energy: -1.240001384480213, parameters: [3.4885308263206483, -4.763005819690338, 6.018909689556745, 4.580231825361526, 5.3376202835626945, 5.503375091367159, 3.201331239688919, -3.165938018064964]
Round num: 2, energy: -1.098690457040345, parameters: [4.488530826320648, -4.763005819690338, 6.018909689556745, 4.580231825361526, 5.3376202835626945, 5.503375091367159, 3.201331239688919, -3.165938018064964]
Round num: 3, energy: -1.56988136797976, parameters: [3.4885308263206483, -3.7630058196903384, 6.018909689556745, 4.580231825361526, 5.3376202835626945, 5.503375091367159, 3.201331239688919, -3.165938018064964]
Round num: 4, energy: -1.5490028651010863, parameters: [3.4885308263206483, -3.7630058196903384, 7.018909689556745, 4.580231825361526, 5.3376202835626945, 5.503375091367159, 3.201331239688919, -3.165938018064964]
Round num: 5, energy: -1.717301148184244, parameters: [3.4885308263206483, -3.7630058196903384, 6.018909689556745, 5.580231825361526, 5.3376202835626945, 5